# Working with Local Dataset

In this tutorial, we will show how to use your own local dataset with the Dataset class. The Dataset class can help you to manage and process your eyetracking data.

## Preparations

We import `pymovements` as the alias `pm` for convenience.

In [ ]:
import pymovements as pm

For demonstration purposes, we will use the raw data provided by the Toy dataset, a sample dataset that comes with *pymovements*.

We will download the resources of this dataset the directory to simulate a local dataset for you.
All downloaded archive files are automatically extracted and then removed.
The directory of the dataset will be `data/ToyDataset`.

After that we won't use the python class anymore and delete the object
(the files on your system will stay in place).
Don't worry if you're confused about these lines as they are not relevant to your use case.

Just keep in mind that we now have some files with gaze data in the directory `data/ToyDataset`.

In [ ]:
toy_dataset = pm.PublicDataset('ToyDataset', root='data/')
toy_dataset.download(remove_finished=True)

del toy_dataset

## Define your Experiment

To use the Dataset class, we first need to create an Experiment instance. This class represents the properties of the experiment, such as the screen dimensions and sampling rate.

In [ ]:
experiment = pm.gaze.Experiment(
    screen_width_px=1280,
    screen_height_px=1024,
    screen_width_cm=38,
    screen_height_cm=30.2,
    distance_cm=68,
    origin='lower left',
    sampling_rate=1000,
)

## Parameters for File Parsing

We also define a `filename_regex` which is a regular expression used to match and extract values from filenames of data files in the dataset. For example, `r'trial_(?P<text_id>\d+)_(?P<page_id>\d+).csv'` will match filenames that follow the pattern `trial_{text_id}_{page_id}.csv` and extract the values of `text_id` and `page_id` for each file.

In [ ]:
filename_regex = r'trial_(?P<text_id>\d+)_(?P<page_id>\d+).csv'

Both values of `text_id` and `page_id` are numeric. We can use a map to define the casting of these values.

In [ ]:
filename_regex_dtypes = {
    'text_id': int,
    'page_id': int,
}

We can also adjust how the CSV files are read.

The `column_map` dictionary maps the original column names in the CSV files to the desired column names. Here the original column names are 'timestamp', **'x'**, and **'y'**, and the desired column names are **'time'**, **'x_right_pix'**, and **'y_right_pix'**, respectively.

In [ ]:
column_map = {
    'timestamp': 'time',
    'x': 'x_right_pix',
    'y': 'y_right_pix',
}

Here, we specify that the separator in the CSV files is a tab (**'\t'**), and we provide the list of original column names and desired column names as the **'columns'** and **'new_columns'** parameters, respectively.

In [ ]:
read_csv_kwargs = {
    'sep': '\t',
    'columns': list(column_map.keys()),
    'new_columns': list(column_map.values()),
}

## Define and load the Dataset

Finaly we create a **Dataset** instance by passing in the root directory, Experiment instance, and other optional parameters such as the filename regular expression and custom CSV reading parameters. The `dataset_dirname`, `raw_dirname`, `preprocessed_dirname`, and `events_dirname` parameters define the names of the directories for the dataset, raw data, preprocessed data, and events data, respectively.

In [ ]:
# Define the path to the dataset directory
dataset_dir = './data/ToyDataset/'

# Set up the Dataset object
dataset = pm.Dataset(
    root=dataset_dir,
    experiment=experiment,
    filename_regex=filename_regex,
    filename_regex_dtypes=filename_regex_dtypes,
    custom_read_kwargs=read_csv_kwargs,
    dataset_dirname='.',
    raw_dirname='raw',
    preprocessed_dirname='preprocessed',
    events_dirname='events',
)

Now we can load the dataset. Here we select a subset including the first page of texts with ID 1 and 2.

In [ ]:
subset = {
    'text_id': [1, 2],
    'page_id': 1,
}

dataset.load(subset=subset)

# Use the Dataset

Once we have created the Dataset instance, we can use its methods to preprocess and analyze data in our local dataset.

In [ ]:
dataset.gaze[0].frame

Here we use the `pix2deg` method to convert the pixel coordinates to degrees of visual angle.

In [ ]:
dataset.pix2deg()

dataset.gaze[0].frame

We can use the `pos2vel` method to calculate the velocity of the gaze position.

In [ ]:
dataset.pos2vel(method='savitzky_golay', window_length=7, polyorder=2)

dataset.gaze[0].frame